In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import os
import shutil

In [ ]:
backup_base_path = '/media/trekker/Extreme SSD/PWGay_backup_12082023/public_html/pwgay/forum'

In [ ]:
avatars_input_dir = os.path.join(backup_base_path, 'images', 'avatars')
avatart_output_dir = '../_tests/assets/avatars/'
if not os.path.exists(avatart_output_dir):
    os.makedirs(avatart_output_dir)

In [ ]:
db = os.environ.get('PSQL_DATABASE')
host = 'localhost'
user ='user'
port = '5432'
psql_engine = f'postgresql://{user}:@{host}:{port}/{db}'

avatar_type :
* 0 -> no avatar
* 1- > [ueser_id]_[some_other_id].jpg, This is then is stored in forum/images/avatars/upload[hash?]_[user_id].jpg. for example:
  * `1764_1392419933.jpg`
  * `1994	1994_1364321301.jpg`
* 2 -> a normal URL, for example:
  * `http://www.skyscrapercity.com/customavatars/avatar170739_1.gif`
  * `http://www.sherv.net/cm/emoticons/penguins/socially-awkward-penguin-meme-smiley-emoticon.png`
  * `http://popdust.presscdn.com/wp-content/uploads/2011/03/ladyGaga_110904581-150x150.jpg`
* 3 -> is always a path relative to forum/images/avatars, for example:
  * `tetes/iconbebe2.gif`
  * `custom/gryf.gif`
  * `600/558.gif`

In [ ]:
custom_avatar_src_path = os.path.join(
        avatars_input_dir,
        'upload'
    )

In [ ]:
custom_avatar_dest_dir = os.path.join(avatart_output_dir, 'upload')
if not os.path.exists(custom_avatar_dest_dir):
    os.makedirs(custom_avatar_dest_dir)

In [ ]:
uploaded_avatars = os.listdir(
    custom_avatar_src_path
)
uploaded_avatars= [
    filename
    for filename in uploaded_avatars
    if len(filename.split('_')) > 1
]

In [ ]:
for avatar_filename in uploaded_avatars:
    user_id = avatar_filename.split('.')[0].split('_')[1]
    ext = avatar_filename.split('.')[1]
    src = os.path.join(custom_avatar_src_path, avatar_filename)
    new_filename = f'{user_id}.{ext}'
    dst = os.path.join(custom_avatar_dest_dir, new_filename)
    shutil.copy(src, dst)

In [ ]:
unique_predefined_avatar_folders_sql = '''
SELECT DISTINCT split_part(user_avatar, '/', 1) as folder from phpbb_users WHERE user_avatar_type = 3
'''
df = pd.read_sql(unique_predefined_avatar_folders_sql, psql_engine)
for f in df['folder'].tolist():
    predef_avatar_folder_dest = os.path.join(avatart_output_dir, 'gallery', f)
    if not os.path.exists(predef_avatar_folder_dest):
        os.makedirs(predef_avatar_folder_dest)

In [ ]:
unique_predefined_avatas_sql = '''
SELECT DISTINCT user_avatar as avatar from phpbb_users WHERE user_avatar_type = 3
'''
df = pd.read_sql(unique_predefined_avatas_sql, psql_engine)

for a in df['avatar'].tolist():
    src = os.path.join(avatars_input_dir, 'gallery', a)
    dst = os.path.join(avatart_output_dir, 'gallery', a)
    shutil.copy(src, dst)

In [ ]:
src